In [1]:
import numpy as np
import pandas as pd
from glob import glob
from matplotlib import pyplot as plt
from matplotlib import patches
import os
from tqdm import tqdm

from scipy.spatial import distance
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.morphology import generate_binary_structure, iterate_structure, binary_erosion

In [2]:
SCREEN_WIDTH = 640
SCREEN_HEIGHT = 480

HUD_WIDTH = 0
HUD_HEIGHT = 96 # 640 x 384
# HUD_HEIGHT = 104 # 640 x 376

ORIGIN_SHAPE = (128, 128)
KERNEL_SHAPE = (20, 12)
TILE_SIZE = 32

In [51]:
data_dir = "data_compressed3/"

In [45]:
def get_players_vpds(rep_name):
    # vpd_paths = glob(data_dir + "*.vpd")
    # print("vpd_paths:",vpd_paths)
    dataframes = [pd.read_csv(_, index_col=None )for _ in vpd_paths]
    num_dataset = len(dataframes)
    
    result = []
    for dataframe in dataframes:
        dataframe = dataframe.set_index('frame')
        dataframe = dataframe.reindex(range(dataframe.tail(1).index[0]))
        dataframe = dataframe.fillna(method='ffill')
        dataframe = dataframe.reset_index()
        dataframe = dataframe.astype(int)
        dataframe = dataframe.set_index('frame')
        result.append(dataframe)
        
    print(result)
    dataframes = pd.concat(result, axis=1)
    dataframes = dataframes.fillna(method='ffill')
    dataframes = dataframes.astype(int)
    
    dataframes_columns = []
    for i in range(num_dataset):
        dataframes_columns.append('vpx_{}'.format(i + 1))
        dataframes_columns.append('vpy_{}'.format(i + 1))
    dataframes.columns = dataframes_columns
    
    return dataframes, num_dataset

In [5]:
def mapping_vpd_into_channel(vpds_tile, kernel_shape=KERNEL_SHAPE, origin_shape=ORIGIN_SHAPE, num_dataset=5):
    frames = np.zeros((np.max(vpds_tile.index.max()) + 1,) + (origin_shape))
    kernel = np.ones(kernel_shape)
    
    for idx, row in vpds_tile.iterrows():
        for i in range(num_dataset):
            frames[idx][row['vpx_{}'.format(i+1)]:row['vpx_{}'.format(i+1)] + kernel.shape[0], row['vpy_{}'.format(i+1)]:row['vpy_{}'.format(i+1)] + kernel_shape[1]] += kernel
    return frames     

In [6]:
def kernel_sum(frame, origin_shape=ORIGIN_SHAPE, kernel_shape=KERNEL_SHAPE):
    width_tile = origin_shape[0] - kernel_shape[0]
    height_tile = origin_shape[1] - kernel_shape[1]

    result = np.zeros((width_tile, height_tile))
    
    for x in range(width_tile):
        for y in range(height_tile):
            result[x][y] = frame[x:x+kernel_shape[0],y:y+kernel_shape[1]].sum()
            # print(x, x+kernel_width, y, y+kernel_height, frame[x:x+kernel_width,y:y+kernel_height].sum())
            # print(frame[x:x+kernel_width,y:y+kernel_height].shape)

    max_vpx = np.argmax(result)
    max_vpx_tile = (max_vpx // height_tile) * TILE_SIZE
    max_vpy_tile = (max_vpx % height_tile) * TILE_SIZE
    
    return result, (max_vpx_tile, max_vpy_tile)

In [7]:
def vpds_tile_into_channel_kernel_sum(channel_vpds):
    res_kernel_sum = []
    max_vp_coord = []
    
    for t, frame in tqdm(enumerate(channel_vpds), total=channel_vpds.shape[0]):
        res, coord = kernel_sum(frame, kernel_shape=KERNEL_SHAPE)
        res_kernel_sum.append(res)
        max_vp_coord.append(coord)
    
    return np.asarray(res_kernel_sum), np.asarray(max_vp_coord)

In [8]:
def find_unique_peaks(peaks, threshhold):
    
    tmp = peaks.reshape(peaks.shape[0], 1, peaks.shape[1])
    distance_each = np.sqrt(np.einsum('ijk, ijk->ij', peaks - tmp, peaks - tmp))
    
    tf_mat = distance_each > threshold    
    tf_mat_except_diagonal = tf_mat[~np.eye(tf_mat.shape[0],dtype=bool)].reshape(tf_mat.shape[0], -1)    
    
    all_true = np.bitwise_and.reduce(tf_mat_except_diagonal, axis=1)
    ## 다른 걸로부터 독립적인 것들
    isolated = np.where(all_true)
    ## 서로 독립적이지 않은 것들
    not_isolated = np.where(~all_true)
    
    ## 서로 독립적이지 않은 것들중 Unique한 것을 찾음
    not_isolated_result = []
    not_isolated_unique = np.unique(tf_mat_except_diagonal[~all_true], axis=0)
    for idx in range(not_isolated_unique.shape[0]):
        niche = np.asarray([(tf_mat_except_diagonal[tf_idx] == not_isolated_unique[idx]).all() for tf_idx in range(tf_mat_except_diagonal.shape[0])])
        not_isolated_result.append(peaks[niche].mean(axis=0).astype(int))

    return peaks[isolated], np.asarray(not_isolated_result)

In [65]:
# path = "./data_compressed/"
replay_file_name_list = os.listdir(data_dir)
print(replay_file_name_list)
for i in replay_file_name_list:
    print(data_dir +i+"/"+i+".rep.vpd")
    vpds, num_dataset = get_players_vpds(data_dir)
    vpds_tile = (vpds / TILE_SIZE).astype(int)
    print("vpd_paths:",vpd_paths)
    channel_vpds = mapping_vpd_into_channel(vpds_tile, num_dataset=1)
    np.save(data_dir + "/" +i +"/vpds_label_masked.npy",channel_vpds)    

['6254']
data_compressed3/6254/6254.rep.vpd
[        vpx   vpy
frame            
0      3456    64
1      3456    64
2      3456    64
3      3456    64
4      3456    64
...     ...   ...
24741  3360  2944
24742  3360  2944
24743  3360  2944
24744  3360  2944
24745  3360  2944

[24746 rows x 2 columns]]
vpd_paths: ['data_compressed3/6254\\6254.rep.vpd']
